In [ ]:
import fastf1
import pandas as pd
import os
from matplotlib import pyplot as plt
import numpy as np
import warnings
from pathlib import Path
import seaborn as sns

warnings.filterwarnings('ignore')

fastf1.Cache.enable_cache(os.getcwd()) # replace with ur own folder path to make a cache for faster loading

# Window size (range of the sliding window used in delta angle calculation)
N = 40 # (40 + 1 + 40  = 81 data points)

# Max and Min delta angle for a data point to be classified as being part of a corner
CORNER_THRESHOLD = 0.5

In [ ]:
year = 2025

gp_events = ['British Grand Prix', 'Belgian Grand Prix', 'Hungarian Grand Prix', 'Dutch Grand Prix', 'Emilia Romagna Grand Prix', 'Monaco Grand Prix']

df_f1 = pd.DataFrame()
drivers = []
teams = []
events = []

for event in gp_events:
    session = fastf1.get_session(year, event, 'R')
    session.load()

    current_drivers = [session.get_driver(num)['Abbreviation'] for num in session.drivers]
    current_teams = [session.get_driver(num)['TeamName'] for num in session.drivers]

    drivers.extend(current_drivers)
    teams.extend(current_teams)
    events.extend([event] * len(current_drivers))

df_f1["Driver"] = drivers
df_f1["Team"] = teams
df_f1["Event"] = events

df_f1.to_csv("f1.csv", index=False)

In [ ]:
race_stats = pd.DataFrame()
race_data = []

for event in df_f1['Event'].unique():
    session = fastf1.get_session(year, event, 'R')
    session.load()

    telemetry = session.laps.pick_driver('VER').get_telemetry()
    telemetry = telemetry[['X', 'Y','Speed']]

    dx1 = telemetry['X'] - telemetry['X'].shift(N)
    dy1 = telemetry['Y'] - telemetry['Y'].shift(N)
    dx2 = telemetry['X'].shift(-N) - telemetry['X']
    dy2 = telemetry['Y'].shift(-N) - telemetry['Y']

    telemetry['Event'] = event

    telemetry['Delta_Angle'] = np.arctan2(dx1*dy2 - dy1*dx2, dx1*dx2 + dy1*dy2)
    telemetry['Delta_Angle'] = telemetry['Delta_Angle'].fillna(0)

    telemetry['IsCorner'] = (telemetry['Delta_Angle'] > CORNER_THRESHOLD) | (telemetry['Delta_Angle'] < -CORNER_THRESHOLD)

    race_data.append(telemetry)

race_stats = pd.concat(race_data, ignore_index=True)
race_stats.to_csv("race_stats.csv", index=False)

In [ ]:
race_stats_csv = pd.read_csv("race_stats.csv")

for event in race_stats_csv["Event"].unique():
    event_data = race_stats_csv[race_stats_csv["Event"] == event]

    plt.figure(figsize=(6.4, 4.8))
    cmap = sns.color_palette('viridis', as_cmap = True)
    norm = plt.Normalize(vmin=race_stats_csv['Delta_Angle'].min(), vmax=race_stats_csv['Delta_Angle'].max())

    sc = sns.scatterplot(data=event_data,
                        x='X', y='Y',
                        hue='Delta_Angle',
                        palette='viridis',
                        hue_norm=norm,
                        legend=False,
                        edgecolor=None)
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])
    cbar = plt.colorbar(sm, ax = plt.gca())

    plt.savefig(f"{event}_deltaAngle.png")
    plt.close()

    plt.figure(figsize=(6.4, 4.8))
    sns.scatterplot(data=event_data,
                    x='X', y='Y',
                    hue='IsCorner',
                    palette={False: "blue", True: "red"},
                    edgecolor=None)

    plt.savefig(f"{event}_isCorner.png")
    plt.close()

In [ ]:
df_f1 = (
    df_f1
    .groupby(["LapNumber", "IsCorner"])
    .agg(
        AvgSpeed=("Speed", "mean"),
        MaxSpeed=("Speed", "max"),
        MinSpeed=("Speed", "min"),
        Count=("Speed", "size")
    )
    .reset_index()
)

df_f1 = df_f1.pivot(
    index="LapNumber", 
    columns="IsCorner", 
    values=["AvgSpeed", "MaxSpeed", "MinSpeed", "Count"]
)

df_f1.columns = [f"{stat}_{'Corner' if corner else 'Straight'}" 
                    for stat, corner in df_f1.columns]

df_f1 = df_f1.reset_index()

df_f1.to_csv("f1.csv")